In [55]:
import pandas as pd
import json
import numpy as np

# Function Definitions and Ranking Loading

In [56]:
def get_highest_cat(cat_list, rankings):
    rankings = [rankings[cat.strip()]['ranking'] for cat in cat_list]
    return cat_list[rankings.index(min(rankings))]

def make_long(df):
    firstyear = df['FirstYear'].tolist()
    lastyear = df['LastYear'].tolist()
    year_nested = [range(x, y+1) for x,y in zip(firstyear, lastyear)]
    year = pd.Series([item for sublist in year_nested for item in sublist])

    is_multiyear = df['YearsActive'] > 1  #  Find all multiyear businesses
    df_try = df[is_multiyear]

    df_long = df.loc[np.repeat(df.index.values, df['YearsActive'])].reset_index(drop=True)

    df_long['Year'] = year.astype(int)
    del df_long['FirstYear']
    del df_long['LastYear']
    del df_long['YearsActive']
    
    df_long.set_index(['DunsNumber', 'Year'], inplace=True)
    
    return df_long

func = lambda x:  get_highest_cat(x.split(','), rankings)

In [57]:
ranking_config = "C:/Users/jc4673/Documents/Columbia/nets_wrangle/category_ranking/ranking_config.json"
with open(ranking_config) as f:
    rankings = json.load(f)

# Open and Examine Store, Load Into Memory

In [58]:
hf_filename = "C:\Users\jc4673\Documents\Columbia\NETS2013_Wrangled\NETS2013_Classifications.h5"

In [59]:
store = pd.HDFStore(hf_filename, mode='a')

In [60]:
store

<class 'pandas.io.pytables.HDFStore'>
File path: C:\Users\jc4673\Documents\Columbia\NETS2013_Wrangled\NETS2013_Classifications.h5
/Classifications            frame_table  (typ->appendable_multi,nrows->58899949,ncols->16,indexers->[index],dc->[FirstYear,DunsNumber,LastYear,Change,Industry,IndustryGroup,Company,TradeName,SIC,Sales,Emp,YearsActive,BEH_LargestPercent,BEH_SIC,Class,BEH_Class])                              
/Locations                  frame_table  (typ->appendable_multi,nrows->59747065,ncols->19,indexers->[index],dc->[FirstYear,DunsNumber,LastYear,BEH_LOC,BEH_ID,Company,PrimAdd,PrimCity,PrimFipsCounty,PrimState,PrimZip,SecAdd,SecCity,SecFipsCounty,SecState,SecZip,Latitude,Longitude,LevelCode])

In [61]:
sub_class = store.select('Classifications', "columns=['DunsNumber', 'FirstYear', 'LastYear', 'Class', 'BEH_Class']")

# Start Creating Subsets and Writing to h5

##  Get Total BEH_Class Counts of all National Businesses

In [64]:
sub_class.reset_index(drop=False, inplace=True)
sub_class['LastYear'] = sub_class['LastYear'].astype(int)

In [65]:
sub_class['YearsActive'] = sub_class['LastYear'] - sub_class['FirstYear']

In [66]:
BEH_counts = sub_class.drop_duplicates('DunsNumber', keep='first')[['DunsNumber', 'BEH_Class']].groupby('BEH_Class').count()

In [67]:
BEH_counts.reset_index(inplace=True)
BEH_counts['BEH_Class'] = BEH_counts['BEH_Class'].apply(func)

In [68]:
BEH_counts = BEH_counts.groupby('BEH_Class').sum()

In [ ]:
#add to the store
store.append('BEH_counts', BEH_counts, data_columns=True)

## Get Counts Yearly of All Non-'Not' Businesses

In [69]:
sub_class_not = sub_class[sub_class['Class'] == 'not']
sub_class = sub_class[sub_class['Class'] != 'not']
grouped = sub_class.groupby(['Class'])

In [70]:
df = pd.DataFrame()
for name, group in grouped:
    long_group = make_long(group).groupby(level=1).count()
    long_group.index = pd.MultiIndex.from_tuples([(name, x) for x in long_group.index])
    df = pd.concat([df, long_group]) #df.append(long_group)

In [71]:
df.index = df.index.set_names(['Class', 'Year'])
df.rename(columns={'Class':  'Count'}, inplace=True)
df['Count'] = df

In [88]:
counts_copy = df

In [89]:
df = df.reset_index(drop=False)
df['Class'] = df['Class'].apply(func)

In [95]:
yearly_counts = df.groupby(['Class', 'Year']).sum()['Count']

In [96]:
store.append('Yearly_Counts', yearly_counts, data_columns=True)

In [97]:
store

<class 'pandas.io.pytables.HDFStore'>
File path: C:\Users\jc4673\Documents\Columbia\NETS2013_Wrangled\NETS2013_Classifications.h5
/Classifications            frame_table  (typ->appendable_multi,nrows->58899949,ncols->16,indexers->[index],dc->[FirstYear,DunsNumber,LastYear,Change,Industry,IndustryGroup,Company,TradeName,SIC,Sales,Emp,YearsActive,BEH_LargestPercent,BEH_SIC,Class,BEH_Class])                              
/Locations                  frame_table  (typ->appendable_multi,nrows->59747065,ncols->19,indexers->[index],dc->[FirstYear,DunsNumber,LastYear,BEH_LOC,BEH_ID,Company,PrimAdd,PrimCity,PrimFipsCounty,PrimState,PrimZip,SecAdd,SecCity,SecFipsCounty,SecState,SecZip,Latitude,Longitude,LevelCode])
/Yearly_Counts              series_table (typ->appendable,nrows->648,ncols->3,indexers->[index],dc->[Class,Year,Count])                                                                                                                                                                       

In [99]:
store.close()

# By State

In [ ]:
infile = "C:\Users\jc4673\Documents\Columbia\NETS2013_Wrangled\NETS2013_Classifications.h5"

store = pd.HDFStore(infile, mode='a')
store

sub_locations = store.select('Locations', where="'PrimState'=['NY', 'CA', 'FL', 'OH', 'VT', 'KS'] & columns=['PrimState']")

sub_locations.index = sub_locations.index.droplevel(1)
idx = np.unique(sub_locations.index, return_index=True)[1]
sub_locations = sub_locations.iloc[idx]

ind = sub_locations.index
sub_class = store.select('Classifications', "columns=['DunsNumber', 'FirstYear', 'LastYear', 'Class', 'BEH_Class']")

store.close()

sub_frame = sub_locations.join(sub_class, how='left')

sub_frame_long = make_long(sub_frame)

In [ ]:
sub_frame.to_hdf("C:\Users\jc4673\Documents\Columbia\NETS2013_Wrangled\long_states.h5", 'long_states', mode='a')